In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-06-02 16:12:31.858508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748880751.877754    8603 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748880751.883926    8603 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748880751.905207    8603 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748880751.905237    8603 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748880751.905239    8603 computation_placer.cc:177] computation placer alr

In [2]:
batch_size = 16

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

# This is the augmentation configuration we will use for testing
# Only Rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# This is a generator that will read pictures found in
# Subfolders of 'data/train', and indefinitely generate
# Bathches of augmented image data
train_generator = train_datagen.flow_from_directory(
    'Data/train', # This is the target directory
    target_size = (150,150), # All images will be resized to 150x150
    batch_size = batch_size,
    class_mode = 'binary') # Since we use binary_crossentropy loss,we need binary labels

# This is a similar generator for validation data
validation_generator = test_datagen.flow_from_directory(
    'Data/valid',
    target_size = (150,150),
    batch_size = batch_size,
    class_mode = 'binary')
    

Found 2003 images belonging to 3 classes.
Found 699 images belonging to 3 classes.


In [3]:
# Create CNN from here
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Activation,Dropout

model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

/home/zahir/Tf_gpu/tfgpu1/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748880756.309780    8603 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
model.add(Flatten()) # This converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [5]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=800 // batch_size
)

Epoch 1/25


/home/zahir/Tf_gpu/tfgpu1/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1748880757.937192    8707 service.cc:152] XLA service 0x73be240055b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748880757.937246    8707 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2050, Compute Capability 8.6
2025-06-02 16:12:37.972090: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748880758.130635    8707 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/125 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.2305 - loss: -12.2322

I0000 00:00:1748880760.851777    8707 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step - accuracy: 0.4736 - loss: -3195119.5000 - val_accuracy: 0.5207 - val_loss: -62369048.0000
Epoch 2/25
  1/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4375 - loss: -73202160.0000

/home/zahir/Tf_gpu/tfgpu1/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4375 - loss: -73202160.0000 - val_accuracy: 0.5207 - val_loss: -64453460.0000
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.4941 - loss: -183357104.0000 - val_accuracy: 0.5207 - val_loss: -880904448.0000
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4375 - loss: -1048281536.0000 - val_accuracy: 0.5207 - val_loss: -898072448.0000
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.4946 - loss: -1600967424.0000 - val_accuracy: 0.5207 - val_loss: -4680244224.0000
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5000 - loss: -4431931904.0000 - val_accuracy: 0.5207 - val_loss: -4725577728.0000
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.4891 - loss: -7116324352.0000 - val_accuracy: 0.5207 - val_loss: -15716811776.0000
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5000 - loss: -18187642880.0000 - val_accuracy: 0.5207 - val_